In [1]:
from deta import Deta
import pandas as pd
import json
from pandas import json_normalize
from collections import Counter
import plotly.graph_objects as go               # pip install plotly
import plotly.express as px                     # pip install plotly-express
from plotly.subplots import make_subplots

In [2]:
DETA_KEY = "c0jo61nr_BhSm5qHprUP75vRSdEmumYfoS1KMCtQW"

# Initialize with a project key
deta = Deta(DETA_KEY)

# This is how to create/connect a database
db = deta.Base("utilities_reports")

In [3]:
def insert_period(period, e_usages, e_costs, w_usages, w_costs, digi_zahirs, digi_anis, 
                  streamyxs, comment):
    """Returns the user on a successful user creation, otherwise raises and error"""
    return db.put({"key": period, "e_usage": e_usages, "e_cost": e_costs, 
                   "w_usage": w_usages, "w_cost": w_costs, "digi_zahir": digi_zahirs, 
                   "digi_ani": digi_anis, "streamyx": streamyxs, "comment": comment})

def fetch_all_periods():
    """Returns a dict of all periods"""
    res = db.fetch()
    return res.items


def get_period(period):
    """If not found, the function will return None"""
    return db.get(period)

In [6]:
# Example Data
#period = "2022_July"
#comment = ""
#e_usages = {"kWh": 340}
#e_costs = {"RM_e": 92.40}
#w_usages = {"m3": 30}
#w_costs = {"RM_w": 10.30}
#digi_zahirs = {"RM_d1": 40}
#digi_anis = {"RM_d2": 20}
#streamyxs = {"RM_s": 147.35}
#insert_period(period, e_usages, e_costs, w_usages, w_costs, digi_zahirs, digi_anis, streamyxs, comment)

In [ ]:
r = db.get('2022_January').values()
r

In [ ]:
#total_e_usage = sum([sum(entry['e_usage'].values()) for entry in entries])
#total_e_cost = sum([sum(entry['e_cost'].values()) for entry in entries])
#total_w_usage = sum([sum(entry['w_usage'].values()) for entry in entries])
#total_w_cost = sum([sum(entry['w_cost'].values()) for entry in entries])

In [ ]:
entries = fetch_all_periods()
df = json.dumps(entries)
df = pd.read_json(df)
df['key'] = df['key'].str.replace('_','/')
#df['key'] = df['key'].str.replace('_',' ')
#df_1 = df['key'].str.split(' ', 1, expand=True)
#df = pd.concat([df,df_1],axis=1)
#df = df[['digi_ani','digi_zahir','e_cost','e_usage','streamyx','w_cost','w_usage',0,1]]
#df = df.rename(columns={0:'year',1:'month'})
#df_2023 = df[df.year == '2023']
#df_2022 = df[df.year == '2022']
#df_2022
df

In [ ]:
df['key']= pd.to_datetime(df['key'])
df_2020 = df[(df['key'] >= "2020-01-01") & (df['key'] <="2020-12-01")]
df_2020 = df_2020.sort_values(by='key')
df_2020

In [ ]:
#df_ecost_2020 = df_2020['e_cost']
#df_ecost_2020

In [ ]:
#for key, value in df_ecost_2020.items():
    #print(key, value)

In [ ]:
df_ecost_2020 = df_2020['e_cost'].map(Counter).groupby(df_2020['key']).sum().reset_index()
df_ecost_2020 = df_ecost_2020['e_cost'].apply(lambda x: x.get('RM_e')).dropna()
print(df_ecost_2020.sum())
df_ecost_2020

In [ ]:
fig_2020 = make_subplots(shared_xaxes=True, specs=[[{'secondary_y': True}]])
fig_2020.add_trace(go.Bar(y = df_ecost_2020,
                          x = ['January','February','March','April','May','June','July',
                              'August','September','October','November','December'], name=''))
fig_2020.update_layout(title_text='Total Utility Cost (RM)',title_x=0.5,
                font=dict(family="Helvetica", size=10),xaxis=dict(tickmode="array"),plot_bgcolor="rgba(0,0,0,0)",yaxis=(dict(showgrid=False)),
                yaxis_title=None,showlegend=False, height=350)
fig_2020.update_annotations(font=dict(family="Helvetica", size=10))
fig_2020.update_xaxes(title_text='', showgrid=False, zeroline=False, showline=True, linewidth=2, linecolor='black')
fig_2020.update_yaxes(showgrid=False, zeroline=False, showline=True, linewidth=2, linecolor='black')
fig_2020